# Track Pattern Recognition using Hough Transform and Tracks Classification

## Introduction

Track pattern recognition is an early step of the reconstruction of data coming from a particle detector. It recognizes tracks among the subdetectors hits. Reconstructed track parameters allow to estimate the particle deviation in a magnetic field, and thus reconstruct its charge and momentum. This information is used for the reconstruction of the decay vertex, to identify the mother particle and for further particle identification.

There is wide variety of the track pattern recognition methods. They differ in how they process the hits, what kind of tracks they are able to recognize and which requirements these tracks should satisfy. Therefore, specifics of an experiment and the detector geometry affect the tracking performance and track pattern recognition methods should be adapted to it accordingly.

In this notebook a track pattern recognition for a 2D detector with circular geometry and uniform magnetic field is considered. The detector schema with hits and tracks of an event is shown in the figure below. The challenge is to recognize tracks of an event with the highest efficiecny. It supposed that one hit can belong to only one track. 

<img src="pic/detector.png" /> <br>

## About this notebook

This notebook demonstrate how Hough Transform and Machine Learning can be used for track pattern recognition. The notebook describes input data, the track pattern recognition method and qualyti metrics, and shows how to use them.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas
import numpy

import user_test_submission as submission

# Input data

In [2]:
data = pandas.read_csv('public_train.csv', index_col=False)
data = data[data['event_id'].values <=10000]

data.head()

,event_id,cluster_id,layer,iphi,x,y
0,3,4,4,53253,53.900430,-265.585662
1,3,1,5,37216,-47.614439,-402.191329
2,3,1,0,7181,-4.253919,-38.767308
3,3,3,2,7937,44.418132,148.499258
4,3,4,0,7657,7.588600,-38.254583


In [3]:
numpy.unique(data['event_id'].values)

array([   3,    4,    7, ..., 9983, 9989, 9995])

## Data Preparation

In [4]:
X = data[[u'event_id', u'layer', u'iphi', u'x', u'y']].values
y = data[[u'event_id', u'cluster_id']].values

# Split Data into Train/Test Samples

In [5]:
from sklearn.model_selection import train_test_split

event_ids = numpy.unique(data['event_id'].values)

event_ids_train, event_ids_test = train_test_split(event_ids, 
                                                   test_size=0.5, 
                                                   random_state=42)

X_train, y_train = X[data['event_id'].isin(event_ids_train), :], y[data['event_id'].isin(event_ids_train)]
X_test, y_test = X[data['event_id'].isin(event_ids_test), 1:], y[data['event_id'].isin(event_ids_test)]

# Hough Transform

## Track Pattern Recognition

In [6]:
from fast_hough import Clusterer

ctr = Clusterer(invr_size=0.05 / 20000., theta_size=2. * numpy.pi / 450.,  min_hits=2, phi_window=0.5)

ctr.fit(X_train, y_train)

In [7]:
%%time
from metrics import predictor

y_pred_test = predictor(ctr, X_test, y_test)

CPU times: user 1min 22s, sys: 243 ms, total: 1min 22s
Wall time: 1min 23s


## Score

In [8]:
score = submission.score_function(y_test, y_pred_test)
score

0.9613754476762905

In [9]:
1. *len(y_pred_test[y_pred_test == -1]) / len(y_pred_test)

0.0008807247106190236

## Additional quality metrics

In [10]:
from metrics import RecognitionQuality

rq = RecognitionQuality(track_eff_threshold=0.999, min_hits_per_track=2)
report_event, report_tracks = rq.calculate(y_test, y_pred_test)

In [11]:
report_event.mean(axis=0)[1:]

ReconstructionEfficiency    0.900547
GhostRate                   0.120949
CloneRate                   0.009059
AvgTrackEfficiency          0.972577
dtype: float64

In [12]:
report_event

,Event,ReconstructionEfficiency,GhostRate,CloneRate,AvgTrackEfficiency
0,7,0.714286,0.285714,0.000000,0.904762
1,26,1.000000,0.000000,0.000000,1.000000
2,44,1.000000,0.000000,0.000000,1.000000
3,48,0.900000,0.100000,0.000000,0.981818
4,63,0.777778,0.222222,0.000000,0.923810
5,75,1.000000,0.000000,0.000000,1.000000
6,83,0.909091,0.181818,0.090909,0.952991
7,85,0.875000,0.125000,0.000000,0.987500
8,87,0.666667,0.166667,0.000000,0.980000
9,93,0.642857,0.500000,0.000000,0.850347
